In [87]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

import glob
from tqdm import tqdm_notebook, tqdm
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPool2D, UpSampling2D, Concatenate
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.models import Model
from keras.utils.vis_utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, Callback
from tensorflow.python.keras.metrics import Metric
import tensorflow as tf

from skimage.io import imread, imshow, concatenate_images
from PIL import Image
import cv2

from albumentations import RandomCrop, HorizontalFlip, VerticalFlip
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))
        #pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

jotting down the path to the dataset !

In [93]:
path = '../input/semantic-drone-dataset/dataset/semantic_drone_dataset/'

!ls {path}

okay so in the above path, we have images and their corresponding labels / masks as well

In [94]:
images_path = glob.glob(os.path.join(path, "original_images/*"))
labels_path = glob.glob(os.path.join(path, "label_images_semantic/*"))

Are they sorted ? Let's print a few and notice the image titles !!!

In [96]:
for i in range(len(images_path)-395):
    print(images_path[i])
    print(labels_path[i])
    #pass

No they are not, so lets sort them first thing !

In [97]:
images_path = sorted(images_path)
labels_path = sorted(labels_path)

Data augmentation !!!
Simply resizing all the images and the corresponding masks

In [ ]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [98]:

def augment_data(images, masks, save_path, augment=False):
    W = 2048
    H = int(W*(3/2))
    for x,y in tqdm(zip(images, masks), total=len(images)):
        name = x.split("/")[-1].split(".")
        image_name = name[0]
        image_extn = name[1]

        name = y.split("/")[-1].split(".")
        mask_name = name[0]
        mask_extn = name[1]       
        
        x = cv2.imread(x, cv2.IMREAD_COLOR)
        x = cv2.resize(x, (W, H))
        y = cv2.imread(y, cv2.IMREAD_COLOR)
        y = cv2.resize(y, (W, H))
        
        if augment == True:
            
            aug = RandomCrop(int(2*H/3), int(2*W/3), always_apply=False, p=1.0)
            augmented = aug(image=x, mask=y)
            x1 = augmented["image"]
            y1 = augmented["mask"]
 
            aug = HorizontalFlip(always_apply=False, p=1.0)
            augmented = aug(image=x, mask=y)
            x2 = augmented["image"]
            y2 = augmented["mask"]
            
            aug = VerticalFlip(always_apply=False, p=1.0)
            augmented = aug(image=x, mask=y)
            x3 = augmented["image"]
            y3 = augmented["mask"] 
            
            save_images = [x, x1, x2, x3]
            save_masks = [y, y1, y2, y3]            
          
        else:
            save_images = [x]
            save_masks = [y]
        
        idx = 0
        for i, m in zip(save_images, save_masks):
            i = cv2.resize(i, (W, H))
            m = cv2.resize(m, (W, H))
            
            tmp_img_name = f"{image_name}_{idx}.{image_extn}"
            tmp_msk_name = f"{mask_name}_{idx}.{mask_extn}" 
            
            image_path = os.path.join(save_path, "images", tmp_img_name)
            mask_path = os.path.join(save_path, "masks", tmp_msk_name)
            
            cv2.imwrite(image_path, i)
            cv2.imwrite(mask_path, m)

            idx+=1

In [99]:
save_path = "./new_data/"

create_dir("./new_data/images/")
create_dir("./new_data/masks/")

to_augment = True

augment_data(images_path, labels_path, save_path, augment=to_augment)


In [100]:
if to_augment:
    images_path = sorted(glob.glob(os.path.join(save_path, "images/*")))
    labels_path = sorted(glob.glob(os.path.join(save_path, "masks/*")))
    print(f"Augmented images:  {len(images_path)} - Augmented masks: {len(labels_path)}")
else:
    images_path = sorted(glob.glob(os.path.join(path, "original_images/*")))
    masks_path = sorted(glob.glob(os.path.join(path, "label_images_semantic/*")))
    print(f"Count of images:  {len(images_path)} - Count of masks: {len(labels_path)}")

Let's look at some images and their corresponding masks !

In [102]:
for i in range(0, 3):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (18, 12))
    aerial_img = imread(images_path[i])
    #print(aerial_img.shape)
    ax1.imshow(aerial_img)
    ax1.set_title('Original Image')
    
    aerial_mask = imread(labels_path[i])
    ax2.imshow(aerial_mask)
    ax2.set_title('Mask of the image')


In [103]:
if to_augment:
    image_path =  os.path.join(save_path, "images/")
    label_path = os.path.join(save_path, "masks/")
else:
    image_path =  os.path.join(path, "original_images/")
    label_path = os.path.join(path, "label_images_semantic/")

name = list()

for dirname, _, filenames in os.walk(image_path):
    for filename in filenames:
        #print(dirname)
        #print(filename)
        name.append(filename.split('.')[0])
        
df_images = pd.DataFrame({'id': name}, index = np.arange(0, len(name)))
df_labels = pd.DataFrame({'id': name}, index = np.arange(0, len(name)))

In [104]:
print(df_images.head())
print(df_labels.head())

In [105]:
Xtrainval, Xtest = train_test_split(df_images['id'], test_size = 0.2, random_state = 19)

print(f"Train Size: {len(Xtrainval)} images")
print(f"Test Size: {len(Xtest)} images")

Xtrain, Xval = train_test_split(Xtrainval, test_size=0.2, random_state=19)

print(f"Val Size: {len(Xval)} images")

In [106]:
Ytrain = Xtrain
Ytest = Xtest
Yval = Xval

In [107]:
if to_augment:
    img_train = [os.path.join(save_path, "images/", f"{name}.jpg") for name in Xtrain]
    mask_train = [os.path.join(save_path, "masks/", f"{name}.png") for name in Ytrain]
    
    img_test = [os.path.join(save_path, "images/", f"{name}.jpg") for name in Xtest]
    mask_test = [os.path.join(save_path, "masks/", f"{name}.png") for name in Ytest]
    
    img_val = [os.path.join(save_path, "images/", f"{name}.jpg") for name in Xval]
    mask_val = [os.path.join(save_path, "masks/", f"{name}.png") for name in Yval]
    
else:
    img_train = [os.path.join(path, "original_images/", f"{name}.jpg") for name in Xtrain]
    mask_train = [os.path.join(path, "label_images_semantic/", f"{name}.png") for name in Ytrain]
    
    img_test = [os.path.join(path, "original_images/", f"{name}.jpg") for name in Xtest]
    mask_test = [os.path.join(path, "label_images_semantic/", f"{name}.png") for name in Ytest]
    
    img_val = [os.path.join(path, "original_images/", f"{name}.jpg") for name in Xval]
    mask_val = [os.path.join(path, "label_images_semantic/", f"{name}.png") for name in Yval]


In [108]:
print(len(img_train))
print(len(img_test))
print(len(img_val))

In [109]:
def conv_block(inputs, filter_count, pool=True, batchnorm = True):
    
    #first layer
    x = Conv2D(filter_count, 3, padding = 'same', kernel_initializer = 'he_normal')(inputs)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    #second layer
    x = Conv2D(filter_count, 3, padding = 'same', kernel_initializer = 'he_normal')(x)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    if pool == True:
        p = MaxPool2D((2, 2))(x)
        return x, p
    else:
        return x


In [110]:
def deconv_block(inputs, concat_layer, filter_count, pool = False):
    u = UpSampling2D((2,2), interpolation='bilinear')(inputs)
    c = Concatenate()([u, concat_layer])
    x = conv_block(c, filter_count, pool = pool, batchnorm = True)
    return u, c, x

In [111]:
def make_me_a_unet(shape, num_classes):
    
    inputs = Input(shape) # 768 x 1152
    
    # Downsampling side of the UNET i.e. the encoder !
    
    x1, p1 = conv_block(inputs, 32, pool=True, batchnorm=True)
    x2, p2 = conv_block(p1, 64, pool=True, batchnorm=True)
    x3, p3 = conv_block(p2, 96, pool=True, batchnorm=True)
    x4, p4 = conv_block(p3, 128, pool=True, batchnorm=True)
    b = conv_block(p4, 256, pool=False, batchnorm=True)
    
    # Upsampling side of the UNET i.e the decoder !
    
    u1, c1, x5 = deconv_block(b, x4, 128)
    u2, c2, x6 = deconv_block(x5, x3, 96)
    u3, c3, x7 = deconv_block(x6, x2, 64)
    u4, c4, x8 = deconv_block(x7, x1, 32)
    
    # The output layer
    
    output = Conv2D(num_classes, 1, padding='same', activation='softmax')(x8)
    
    #softmax for multiclass classification, num_classes = 23 !
    
    return Model(inputs, output)

In [112]:
#Setting some parameters

H = 768 
W = int(H*(3/2)) 
num_classes = 23

In [113]:
model = make_me_a_unet((W, H, 3), num_classes)

In [114]:
model.summary()

In [115]:
def read_image(x):
    x = cv2.imread(x, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x

In [116]:
def read_mask(x):
    x = cv2.imread(x, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (W, H))
    x = x.astype(np.int32)
    return x

In [117]:
def tf_dataset(x,y, batch=1):
    dataset = tf.data.Dataset.from_tensor_slices((x,y))
    dataset = dataset.shuffle(buffer_size=100)
    dataset = dataset.map(preprocess)
    dataset = dataset.batch(batch)
    dataset = dataset.repeat()
    dataset = dataset.prefetch(2)
    return dataset

In [118]:
def preprocess(x,y):
    def f(x,y):
        x = x.decode()
        y = y.decode()
        image = read_image(x)
        mask = read_mask(y)
        return image, mask
    
    image, mask = tf.numpy_function(f,[x,y],[tf.float32, tf.int32])
    mask = tf.one_hot(mask, num_classes, dtype=tf.int32)
    image.set_shape([H, W, 3])
    mask.set_shape([H, W, num_classes])
    return image, mask

In [119]:
# Seeding
np.random.seed(42)
tf.random.set_seed(42)
                 
shape = (H, W, 3) 

# Hyperparameters
lr = 1e-4 # Learning rate of Adam optimizer
batch_size = 4
epochs = 30

In [120]:
# Model
model = make_me_a_unet(shape, num_classes)
model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(lr), metrics=['accuracy'])

train_dataset = tf_dataset(img_train, mask_train, batch = batch_size)
valid_dataset = tf_dataset(img_val, mask_val, batch = batch_size)

train_steps = len(img_train)//batch_size
valid_steps = len(img_val)//batch_size

callbacks = [
    ModelCheckpoint("model.h5", verbose=1, save_best_model=True),
    ReduceLROnPlateau(monitor='val_loss', patience=5, factor=0.1, verbose=1, min_lr=1e-5),
    EarlyStopping(monitor='val_loss', patience=5, verbose=1)
]

## Train the model
model.fit(train_dataset,
          steps_per_epoch=train_steps,
          validation_data=valid_dataset,
          validation_steps=valid_steps,
          epochs=epochs,
          callbacks=callbacks
         )

In [ ]:
#from keras.models import load_model
#model = load_model('model.h5')

In [121]:
## Plot accuracy and loss

train_loss = model.history.history['loss']
val_loss   = model.history.history['val_loss']
train_acc  = model.history.history['accuracy']
val_acc    = model.history.history['val_accuracy']

In [122]:
# summarize history for accuracy
plt.plot(model.history.history['accuracy'])
plt.plot(model.history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [123]:
# summarize history for loss
plt.plot(model.history.history['loss'])
plt.plot(model.history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [127]:
## Prediction

os.makedirs('./results')
np.random.seed(42)
tf.random.set_seed(42)


In [126]:
# Saving the masks
for x, y in tqdm(zip(img_test, mask_test), total=len(img_test)):
    name = x.split("/")[-1]
    
    ## Read image
    x = cv2.imread(x, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)

    ## Read mask
    y = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
    y = cv2.resize(y, (W, H))
    
    y = np.expand_dims(y, axis=-1) #(384,256,1)
    
    y = y * (255/num_classes)
    y = y.astype(np.int32)
    y = np.concatenate([y, y, y], axis=2)
    
    ## Prediction
    p = model.predict(np.expand_dims(x, axis=0))[0]
    p = np.argmax(p, axis=-1)
    
    p = np.expand_dims(p, axis=-1)  
    
    p = p * (255/num_classes)
    p = p.astype(np.int32)
    p = np.concatenate([p, p, p], axis=2)
      
    cv2.imwrite(f"./results/{name}", p)

In [128]:
# From the test set, take only images that represent the ones in the original dataset and not those are obtained from the data augmentation.
# (they have _0 in the name)

image_list = []
mask_list = []

for x,y in tqdm(zip(img_test, mask_test), total=len(img_test)):
    name = x.split("/")[-1]
    image_name = name
    #image_name = name
    #print(image_name)
    name = y.split("/")[-1]
    mask_name = name
    #mask_name = name
    #print(mask_name)
    
    #if image_name == '0':
    image_list.append(x)
    mask_list.append(y)

In [129]:
## Plot a few images to verify the accuracy in the predictions

img_selection = image_list[0:10]
mask_selection = mask_list[0:10]

for img, mask in zip(img_selection, mask_selection):
    name = img.split("/")[-1]
    x = cv2.imread(img, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))

    y = cv2.imread(mask, cv2.IMREAD_GRAYSCALE)
    y = cv2.resize(y, (W, H))

    p = cv2.imread(f"./results/{name}", cv2.IMREAD_GRAYSCALE)
    p = cv2.resize(p, (W, H))

    #Plotto le tre immagini
    fig, axs = plt.subplots(1, 3, figsize=(18, 12), constrained_layout=True)
    
    axs[0].imshow(x, interpolation = 'nearest')
    axs[0].set_title('image')
    axs[0].grid(False)

    axs[1].imshow(y, interpolation = 'nearest')
    axs[1].set_title('mask')
    axs[1].grid(False)

    axs[2].imshow(p)
    axs[2].set_title('prediction')
    axs[2].grid(False)
